In [2]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [17]:
TRAIN_DATA_PATH = './train.csv'
TEST_DATA_PATH = './test.csv'
SUBMIT_DATA_PATH = './sample_submission.csv'

In [4]:
train = pd.read_csv(TRAIN_DATA_PATH)
test = pd.read_csv(TEST_DATA_PATH)

In [5]:
x_train = train.drop(['target','ID_code'], axis=1)
y_train = train['target'].values
use_cols = x_train.columns.values

In [6]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

list_auc_score = []
list_logloss_score = []

for trn_idx, val_idx in cv.split(x_train, y_train):
    trn_x = x_train.iloc[trn_idx, :]
    val_x = x_train.iloc[val_idx, :]
    trn_y = y_train[trn_idx]
    val_y = y_train[val_idx]
    
    clf = LogisticRegression(random_state=1)
    clf.fit(trn_x, trn_y)
    pred = clf.predict_proba(val_x)[:, 1]
    
    sc_logloss = log_loss(val_y, pred)
    sc_auc = roc_auc_score(val_y, pred)
    
    list_logloss_score.append(sc_logloss)
    list_auc_score.append(sc_auc)

In [12]:
clf = LogisticRegression(random_state=1)
clf.fit(x_train, y_train)

x_test = test[use_cols]
pred_test = clf.predict_proba(x_test)[:, 1]

In [20]:
df_submit = pd.read_csv(SUBMIT_DATA_PATH)
df_submit['target'] = pred_test
df_submit.to_csv('LogisticRegression.csv', index=False)